In [ ]:
# Все библиотеки

import pandas as pd
from scipy.stats import ttest_ind, levene, mannwhitneyu,shapiro, \
    f_oneway, kruskal, chi2_contingency, fisher_exact
import numpy as np
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
import seaborn as sns
from sklearn.metrics import roc_auc_score, roc_curve
import pingouin as pg
import warnings

warnings.filterwarnings('ignore')

In [ ]:
income = pd.read_csv('заработок бобров.csv', encoding='utf-8', sep=';')
purs = pd.read_csv('покупки.csv', encoding='utf-8', sep=';')
staff = pd.read_excel('сотрудники.xlsx')

In [ ]:
def obr(df):
    df.columns = df.columns.str.replace(' ', '_').str.lower().str.replace('.', '')
    return df

In [ ]:
income = obr(income)
purs = obr(purs)
staff = obr(staff)

In [161]:
income

,код_сотрудника,департамент,сумма_вознаграждения,название_миссии,контейнер,дата,комментарий,дней_назад,стаж_в_днях,стаж_на_момент_заработка
0,SU147663,ДБП.группа консультантов по автоматизации бизн...,50.0,Ты вовремя списал затраченное время на задачи ...,Проектный опыт,2024-02-06,За неделю 29.01-04.02,425,1528.0,1299.0
1,SU76490,ДВА.группа автоматизации производственных проц...,50.0,Ты вовремя списал затраченное время на задачи ...,Проектный опыт,2024-01-24,За неделю 15.01-21.01,438,3468.0,3226.0
2,SU69215,ДИРС.группа инженеров по ИТ-инфраструктуре и м...,50.0,Ты вовремя списал затраченное время на задачи ...,Проектный опыт,2024-07-29,За неделю 15.07-21.07,251,3983.0,3928.0
3,SU4528,Служба охраны.группа обеспечения контрольно-пр...,5.0,Ты угадал коллегу в Jive с первого раза,Корпоративная жизнь КРОК,2024-05-02,NaN,339,5574.0,5431.0
4,SU163517,ДРП.отдел кадров,5.0,Ты угадал коллегу в Jive с первого раза,Корпоративная жизнь КРОК,2024-09-06,NaN,212,1223.0,1207.0
...,...,...,...,...,...,...,...,...,...,...
94877,SU190343,ДВА.группа разработки инфраструктурных приложений,5.0,Ты угадал коллегу в Jive с первого раза,Корпоративная жизнь КРОК,2024-04-11,NaN,360,780.0,616.0
94878,SU44113,ДВА.группа разработки 1С,50.0,Ты вовремя списал затраченное время на задачи ...,Проектный опыт,2024-10-02,За неделю 16.09 - 22.09,186,4867.0,4877.0
94879,SU173704,ДТК.Цифровое производство.группа инженеров циф...,50.0,Ты вовремя списал затраченное время на задачи ...,Проектный опыт,2024-02-06,За неделю 29.01-04.02,425,NaN,NaN
94880,SU43534,"ДРПО.Практика ""Роботехника"".группа разработки ...",200.0,"Плюс один год, как ты в команде КРОК!",Я и КРОК,2024-03-04,NaN,398,4911.0,4709.0


In [ ]:
income.drop_duplicates(inplace=True)

In [160]:
missions_to_drop = [
    'тренер',
    'Croc sport',
    'Croc sport( 4 уровень)',
    'миссия best hunter',
    'миссия креативный класс',
    'путешественник КРОК',
    'развитие бренда работодателя',
    'продвижение IT-профессий'
]

comments_to_drop = ['ошибочно начислены']

income = income[~income['название_миссии'].str.lower().isin([m.lower() for m in missions_to_drop])]
income = income[~income['комментарий'].str.lower().isin([m.lower() for m in comments_to_drop])]

In [ ]:
avg_tasks_per_person = income.groupby('код_сотрудника')['сумма_вознаграждения'].sum().mean()
print(f"📈 Среднее количество выполненных задач на одного человека: {avg_tasks_per_person:.2f}")

In [162]:
good = [
'Ты вовремя списал затраченное время на задачи и проекты',
'Ты вовремя подписал документы по кадровым движениям',
'Креативный класс',
'Ты проверил информацию при обновлении проектного опыта (для юристов, за 1 проект)',
'Ты прошел все онлайн-курсы для новых сотрудников',
'Ты проверил информацию при сборе проектного опыта (для маркетологов, специалистов службы качества, за 1 проект)',
'Ты заполнил проектный опыт (для менеджеров проекта, за 1 проект)',
'Ты повторно проверил информацию при обновлении проектного опыта (для юристов, за 1 проект)',
'Ты повторно проверил информацию при обновлении проектного опыта (для маркетологов, специалистов службы качества, за 1 проект)',
'Миссия Креативный класс',
'Ты обновил полностью проектный опыт (для в менеджера проекта, за 1 проект)',
'Ты успешно сдал экзамен*',
'Ты прошел программу адаптации для аналитиков',
'Ты сдал экзамен* на 100%'
]


In [163]:
from scipy.stats import wilcoxon


def good_count(missions):
    return sum(mission in good for mission in missions)


def effectivnost(missions):
    total = len(missions)
    if total == 0:
        return 0.0
    return good_count(missions) / total


def sravnenie(df, missions):
    df['дата'] = pd.to_datetime(df['дата'])

    results = []
    for employee_id, group in df[['код_сотрудника', 'дата', 'название_миссии']].groupby('код_сотрудника'):
        group = group.sort_values('дата')
        top = group[group['название_миссии'].str.contains('|'.join(missions), na=False)]
        if len(top) == 0:
            continue

        if not top.empty:
            top_data = top['дата'].iloc[0]
        else:
            top_data = pd.Timestamp.max

        missions_before = group[group['дата'] < top_data]['название_миссии']
        missions_after = group[group['дата'] >= top_data]['название_миссии']

        start_date = pd.to_datetime("2023-09-02")
        before_diff = (top_data - start_date).days

        end_date = pd.to_datetime("2024-10-15")
        after_diff = (end_date - top_data).days

        k_before = len(missions_before)/before_diff
        k_after = len(missions_after)/after_diff

        good_k_before = good_count(missions_before)/before_diff
        good_k_after = good_count(missions_after)/after_diff

        eff_before = effectivnost(missions_before)
        eff_after = effectivnost(missions_after)

        results.append({
            'код_сотрудника': employee_id,
            'эффективность_до': eff_before,
            'эффективность_после': eff_after,
            'k_before': k_before,
            'k_after': k_after,
            'good_k_before': good_k_before,
            'good_k_after': good_k_after,
            'missions_before': missions_before.shape[0],
            'missions_after': missions_after.shape[0]
        })

    eff_df = pd.DataFrame(results)

    valid_eff = eff_df.dropna(subset=['эффективность_до', 'эффективность_после'])

    stat, p_value = wilcoxon(valid_eff['эффективность_до'], valid_eff['эффективность_после'])
    stat2, p_value2 = wilcoxon(valid_eff['k_before'], valid_eff['k_after'])
    stat3, p_value3 = wilcoxon(valid_eff['good_k_before'], valid_eff['good_k_after'])

    print(f"{missions} :\nПРОЦЕНТЫ\nWilcoxon test: статистика = {stat}, p-value = {p_value:.5f}")

    if p_value < 0.05:
        print("📈 Есть статистически значимая разница между эффективностью ДО и ПОСЛЕ ключевых значений (p < 0.05)")
    else:
        print("📉 Статистически значимой разницы не найдено (p ≥ 0.05)")


    print()
    print(f"до : {eff_df['эффективность_до'].median()}")
    print(f"после : {eff_df['эффективность_после'].median()}")
    print()

    print(f"КОЛ-ВО ЗАДАЧ В ДЕНЬ\nWilcoxon test: статистика = {stat2}, p-value = {p_value2:.5f}")

    if p_value2 < 0.05:
        print("📈 Есть статистически значимая разница между эффективностью ДО и ПОСЛЕ ключевых значений (p < 0.05)")
    else:
        print("📉 Статистически значимой разницы не найдено (p ≥ 0.05)")

    print()
    print(f"до : {eff_df['k_before'].median()}")
    print(f"после : {eff_df['k_after'].median()}")
    print()

    print(f"КОЛ-ВО КРУТЫХ ЗАДАЧ В ДЕНЬ\nWilcoxon test: статистика = {stat3}, p-value = {p_value3:.5f}")

    if p_value3 < 0.05:
        print("📈 Есть статистически значимая разница между эффективностью ДО и ПОСЛЕ ключевых значений (p < 0.05)")
    else:
        print("📉 Статистически значимой разницы не найдено (p ≥ 0.05)")

    print()
    print(f"до : {eff_df['good_k_before'].median()}")
    print(f"после : {eff_df['good_k_after'].median()}")

    print()
    print(f"среднее кол-во миссий до: {eff_df['missions_before'].median()}")
    print(f"среднее кол-во миссий после: {eff_df['missions_after'].median()}")


TheBest = ['HiPro', 'HiPo', 'Креативный класс', 'Звезда департамента']
sravnenie(income, TheBest)

['HiPro', 'HiPo', 'Креативный класс', 'Звезда департамента'] :
ПРОЦЕНТЫ
Wilcoxon test: статистика = 221975.0, p-value = 0.06242
📉 Статистически значимой разницы не найдено (p ≥ 0.05)

до : 0.2
после : 0.25

КОЛ-ВО ЗАДАЧ В ДЕНЬ
Wilcoxon test: статистика = 26403.0, p-value = 0.00000
📈 Есть статистически значимая разница между эффективностью ДО и ПОСЛЕ ключевых значений (p < 0.05)

до : 0.024096385542168676
после : 0.07920792079207921

КОЛ-ВО КРУТЫХ ЗАДАЧ В ДЕНЬ
Wilcoxon test: статистика = 51020.5, p-value = 0.00000
📈 Есть статистически значимая разница между эффективностью ДО и ПОСЛЕ ключевых значений (p < 0.05)

до : 0.005780346820809248
после : 0.018518518518518517

среднее кол-во миссий до: 5.0
среднее кол-во миссий после: 11.0


In [ ]:
effectivnost(income['название_миссии'])

In [ ]:
income[income['название_миссии'].isin(good)]['сумма_вознаграждения'].sum()/income['сумма_вознаграждения'].sum()

In [ ]:
income['дата'] = pd.to_datetime(income['дата'], format='%d.%m.%Y')
today = pd.Timestamp.today().normalize()
income['дней_назад'] = (today - income['дата']).dt.days

In [ ]:
income['стаж_на_момент_заработка'] = income['стаж_на_момент_заработка'] - income['стаж_на_момент_заработка'].min()
count_incomes_by_stazh = income[income['название_миссии'].isin(good)][['стаж_на_момент_заработка', 'сумма_вознаграждения']].dropna().pivot_table(values='сумма_вознаграждения', index='стаж_на_момент_заработка', aggfunc='mean')
count_incomes_by_stazh = count_incomes_by_stazh.reset_index()
count_incomes_by_stazh['стаж (по квартилям)'] = count_incomes_by_stazh['стаж_на_момент_заработка'] // 90

In [ ]:
count_incomes_by_stazh

In [154]:
purs

,дата_оформления_заказа,код_сотрудника,департамент,наименование_товара,стоимость_в_валюте,скидка,количество,категория,статус_товара,дней_назад,стаж_в_днях,стаж_на_момент_покупки
0,2024-10-15 17:16:00,SU162966,ДРП.HR-партнеры,Бобер-антистресс,250,0,1,Мерч и сувениры КРОК,закончился,173,1237.0,1064.0
1,2024-10-15 16:34:00,SU199359,Склад,Welcome-набор с чехлом и футболкой,0,0,1,Без категории,закончился,173,93.0,-80.0
2,2024-10-15 16:31:00,SU180088,Отдел дизайна,Олимпийка SOFT `24,0,100,1,В стиле КРОК,на витрине,173,1022.0,849.0
3,2024-10-15 16:02:00,SU198080,ДИРС.группа пресейл-инженеров,Побег из города,0,0,1,Без категории,закончился,173,280.0,107.0
4,2024-10-15 15:56:00,SU198606,ДИРС.группа инженеров по инфраструктурному ПО,Бобер-антистресс,250,0,1,Мерч и сувениры КРОК,закончился,173,200.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...
37184,2019-09-23 07:16:00,SU116489,ДВА.центральная служба технической поддержки,"Термокружки ""CROC IT NOW"" на выбор",700,0,1,В стиле КРОК,закончился,2022,NaN,NaN
37185,2019-09-22 16:14:00,SU69130,Отдел дизайна,Фикус,1500,0,1,Прокачай эффективность,закончился,2023,3971.0,1948.0
37186,2019-09-22 16:11:00,SU69130,Отдел дизайна,"Термокружки ""CROC IT NOW"" на выбор",700,0,1,В стиле КРОК,закончился,2023,3971.0,1948.0
37187,2019-09-22 16:10:00,SU69130,Отдел дизайна,"Журнал ""Harvard Bisiness Review""",800,0,1,Сервис,закончился,2023,3971.0,1948.0


In [156]:
purs['департамент'].value_counts()

департамент
ПД.группа комплексного сопровождения проектов                                  573
ДПМП.группа продаж                                                             572
ДИМС.Группа электроснабжения и климатических систем                            550
ДИРС.группа инженеров по вычислительной инфраструктуре                         536
ДРП.отдел кадров                                                               533
                                                                              ... 
ДИРС.группа эксплуатации облачных сервисов                                       1
ДВС.группа сопровождения ИТ-инфраструктур                                        1
ДРПО.Цифровые решения для промышленности.группа поддержки продаж                 1
ДРПО.КИС.Практика "Медицинские информационные системы".группа разработчиков      1
ООО "Рэд Лаб"                                                                    1
Name: count, Length: 371, dtype: int64

In [ ]:
waiting_time = 7

def check_purchase_after_best_mission(income, purs, missions):
    best_missions = income[income['название_миссии'].str.contains('|'.join(missions), na=False)]
    # best_missions = income

    result = []

    for employee_id, group in best_missions.groupby('код_сотрудника'):
        mission_date = group['дата'].min()

        purchases = purs[purs['код_сотрудника'] == employee_id]

        purchases_after = purchases[
            (purchases['дата_оформления_заказа'] > mission_date) &
            (purchases['дата_оформления_заказа'] <= mission_date + pd.Timedelta(days=waiting_time))
        ]

        result.append({
            'код_сотрудника': employee_id,
            'дата_миссии': mission_date,
            'есть_покупка_в_неделю': not purchases_after.empty
        })

    result_df = pd.DataFrame(result)

    total = result_df.shape[0]
    count_yes = result_df['есть_покупка_в_неделю'].sum()
    print(f"🧾 Сотрудников с TheBest-миссией: {total}")
    print(f"🛒 Из них совершили покупку в течение недели: {count_yes} ({(count_yes / total) * 100:.2f}%)")

    return result_df


def check_purchase_after_best_mission2(income, purs, missions):
    # best_missions = income[income['название_миссии'].str.contains('|'.join(missions), na=False)]
    best_missions = income

    result = []

    for employee_id, group in best_missions.groupby('код_сотрудника'):
        mission_date = group['дата'].min()

        purchases = purs[purs['код_сотрудника'] == employee_id]

        purchases_after = purchases[
            (purchases['дата_оформления_заказа'] > mission_date) &
            (purchases['дата_оформления_заказа'] <= mission_date + pd.Timedelta(days=waiting_time))
        ]

        result.append({
            'код_сотрудника': employee_id,
            'дата_миссии': mission_date,
            'есть_покупка_в_неделю': not purchases_after.empty
        })

    result_df = pd.DataFrame(result)

    total = result_df.shape[0]
    count_yes = result_df['есть_покупка_в_неделю'].sum()
    print(f"🧾 Сотрудников с миссией: {total}")
    print(f"🛒 Из них совершили покупку в течение недели: {count_yes} ({(count_yes / total) * 100:.2f}%)")

    return result_df


print("крутые сотрудники:")
purchase_result = check_purchase_after_best_mission(income, purs, TheBest)
print("просто сотрудники:")
purchase_result2 = check_purchase_after_best_mission2(income, purs, TheBest)

In [ ]:
from scipy.stats import wilcoxon

def purchase_probability_comparison(income, purs, the_best):
    income['дата'] = pd.to_datetime(income['дата'])
    purs['дата_оформления_заказа'] = pd.to_datetime(purs['дата_оформления_заказа'])

    result = []

    for employee_id, group in income.groupby('код_сотрудника'):
        group = group.sort_values('дата')
        top_missions = group[group['название_миссии'].str.contains('|'.join(the_best), na=False)]
        if top_missions.empty:
            continue

        top_date = top_missions['дата'].iloc[0]

        before_missions = group[(group['дата'] < top_date) & ~group['название_миссии'].isin(the_best)]
        after_missions = group[(group['дата'] >= top_date) & ~group['название_миссии'].isin(the_best)]

        def count_purchases(mission_dates):
            count = 0
            total = len(mission_dates)
            for date in mission_dates:
                purchases = purs[
                    (purs['код_сотрудника'] == employee_id) &
                    (purs['дата_оформления_заказа'] > date) &
                    (purs['дата_оформления_заказа'] <= date + pd.Timedelta(days=7))
                ]
                if not purchases.empty:
                    count += 1
            return count, total

        before_count, before_total = count_purchases(before_missions['дата'])
        after_count, after_total = count_purchases(after_missions['дата'])

        if before_total > 0 and after_total > 0:
            result.append({
                'код_сотрудника': employee_id,
                'доля_до': before_count / before_total,
                'доля_после': after_count / after_total
            })

    result_df = pd.DataFrame(result)

    stat, p_value = wilcoxon(result_df['доля_до'], result_df['доля_после'])

    print("📊 Wilcoxon test по долям миссий с покупками:")
    print(f"статистика = {stat}, p-value = {p_value:.5f}")
    if p_value < 0.05:
        print("✅ Есть статистически значимая разница (p < 0.05)")
    else:
        print("❌ Нет статистически значимой разницы (p ≥ 0.05)")

    print()
    print(f"Медианная доля покупок ДО: {result_df['доля_до'].mean():.3f}")
    print(f"Медианная доля покупок ПОСЛЕ: {result_df['доля_после'].mean():.3f}")

    return result_df


result_df = purchase_probability_comparison(income, purs, TheBest)

In [ ]:
count_buys_by_stazh = purs[['стаж_на_момент_покупки', 'стоимость_в_валюте']].dropna().pivot_table(values='стоимость_в_валюте', index='стаж_на_момент_покупки', aggfunc='count')
count_buys_by_stazh = count_buys_by_stazh.reset_index()
count_buys_by_stazh['стаж_на_момент_покупки'] = count_buys_by_stazh['стаж_на_момент_покупки'] - count_buys_by_stazh['стаж_на_момент_покупки'].min()

In [ ]:
count_buys_by_stazh['стаж (в кварталах)'] = count_buys_by_stazh['стаж_на_момент_покупки'] // 90

In [ ]:
count_buys_by_stazh.columns = count_buys_by_stazh.columns.str.replace('стоимость_в_валюте', 'Количество заказов')

In [ ]:
count_buys_by_stazh

In [ ]:
purs['дата_оформления_заказа'] = pd.to_datetime(purs['дата_оформления_заказа'], format='%d.%m.%Y %H:%M')
today = pd.Timestamp.today().normalize()
purs['дней_назад'] = (today - purs['дата_оформления_заказа'].dt.normalize()).dt.days

In [ ]:
staff

In [ ]:
def f(x):
    return income[income['код_сотрудника'] == x][income['название_миссии'].isin(TheBest)].shape[0]


staff['count_the_best'] = staff['внешний_код'].apply(lambda x: f(x))

In [ ]:
staff['count_the_best'].value_counts().reset_index()

In [ ]:
for i in staff['является_рм/тл'].value_counts().head(3).reset_index()['является_рм/тл'].unique():
    print(i)
    sravnenie(income[income['код_сотрудника'].isin(staff[staff['является_рм/тл'] == i]['код_сотрудника'].tolist())], TheBest)

In [ ]:
sravnenie(income[income['код_сотрудника'].isin(staff[staff['пол'] == 'Женский']['код_сотрудника'].tolist())], TheBest)

In [ ]:
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

income['дата'] = pd.to_datetime(income['дата'])
grouped = income.groupby('код_сотрудника')['название_миссии'].agg(list).reset_index()
grouped['кол_миссий'] = grouped['название_миссии'].apply(len)
grouped['эффективность'] = grouped['название_миссии'].apply(lambda x: sum(m in good for m in x) / len(x) if len(x) > 0 else 0)

corr, p_value = spearmanr(grouped['кол_миссий'], grouped['эффективность'])

print(f"Корреляция Спирмена между количеством миссий и эффективностью: {corr:.3f}, p-value = {p_value:.5f}")
if p_value < 0.05:
    print("📊 Есть статистически значимая связь между количеством миссий и эффективностью")
else:
    print("😐 Нет статистически значимой связи между количеством миссий и эффективностью")

plt.scatter(grouped['кол_миссий'], grouped['эффективность'], alpha=0.5)
plt.xlabel('Количество миссий')
plt.ylabel('Эффективность')
plt.title('Зависимость эффективности от количества миссий')
plt.grid(True)
plt.show()


In [ ]:
from scipy.stats import wilcoxon

def good_count(missions):
    return sum(mission in good for mission in missions)


def sravnenie2(df, missions):
    df['дата'] = pd.to_datetime(df['дата'])

    results = []
    for employee_id, group in df[['код_сотрудника', 'дата', 'название_миссии']].groupby('код_сотрудника'):
        group = group.sort_values('дата')
        top = group[group['название_миссии'].str.contains('|'.join(missions), na=False)]
        if not top.empty:
            top_data = top['дата'].iloc[0]
        else:
            top_data = pd.Timestamp.max

        missions_before = group[group['дата'] < top_data]['название_миссии']
        missions_after  = group[group['дата'] >= top_data]['название_миссии']

        start_date = pd.to_datetime("2023-09-02")
        before_diff = max((top_data - start_date).days, 1)
        end_date = pd.to_datetime("2024-10-15")
        after_diff = max((end_date - top_data).days, 1)

        good_k_before = good_count(missions_before) / before_diff
        good_k_after = good_count(missions_after) / after_diff

        results.append({
            'код_сотрудника': employee_id,
            'good_k_before': good_k_before,
            'good_k_after': good_k_after
        })

    eff_df = pd.DataFrame(results)

    stat, p_value = wilcoxon(eff_df['good_k_before'], eff_df['good_k_after'])

    print(f"Wilcoxon test: статистика = {stat}, p-value = {p_value:.5f}")
    if p_value < 0.05:
        print("📈 Есть статистически значимая разница ДО и ПОСЛЕ по эффективности (good/day)")
    else:
        print("📉 Разницы нет (p ≥ 0.05)")

    print()
    print(f"Средняя эффективность до: {eff_df['good_k_before'].mean():.5f}")
    print(f"Средняя эффективность после: {eff_df['good_k_after'].mean():.5f}")


In [ ]:
TheBest = ['HiPro', 'HiPo', 'Креативный класс', 'Звезда департамента']
sravnenie2(income, TheBest)

In [ ]:
def f(s):
    return purs[purs['код_сотрудника'] == s['код_сотрудника']]['стоимость_в_валюте'].sum() / income[income['код_сотрудника'] == s['код_сотрудника']]['сумма_вознаграждения'].sum()


staff['k'] = staff.apply(lambda x: f(x), axis=1)

In [ ]:
staff

In [ ]:
staff['код_сотрудника'] = staff['внешний_код']

In [ ]:
purs = purs.merge(
    staff[['код_сотрудника', 'стаж_в_днях']],
    on='код_сотрудника',
    how='left'
)
purs['стаж_на_момент_покупки'] = purs['стаж_в_днях'] - purs['дней_назад']


income = income.merge(
    staff[['код_сотрудника', 'стаж_в_днях']],
    on='код_сотрудника',
    how='left'
)
income['стаж_на_момент_заработка'] = income['стаж_в_днях'] - income['дней_назад']

In [ ]:
import re


def parse_stazh_to_days(text):
    years = months = days = 0

    year_match = re.search(r'(\d+)\s*г\.', text)
    if year_match:
        years = int(year_match.group(1))

    month_match = re.search(r'(\d+)\s*мес\.', text)
    if month_match:
        months = int(month_match.group(1))

    day_match = re.search(r'(\d+)\s*дн\.', text)
    if day_match:
        days = int(day_match.group(1))

    total_days = years * 360 + months * 30 + days
    return total_days

staff['стаж_в_днях'] = staff['стаж_фактический_по_компании'].apply(parse_stazh_to_days)

In [ ]:
income.to_csv("Датасет без миссий с нулевой прибылью.csv", sep=';')

In [ ]:
income

In [ ]:
# Убедимся, что дата в правильном формате
income['дата'] = pd.to_datetime(income['дата'])

# Получаем все уникальные месяцы в формате "YYYY-MM"
unique_months = income['дата'].dt.to_period('M').sort_values().unique()

# Создаем список всех месяцев между минимальной и максимальной датой
full_range = pd.period_range(start=income['дата'].min(), end=income['дата'].max(), freq='M')

# Проверяем, есть ли пропущенные месяцы
missing_months = full_range.difference(unique_months)

if len(missing_months) == 0:
    print("✅ Все месяцы присутствуют в данных!")
else:
    print("⚠️ Отсутствуют следующие месяцы:")
    for month in missing_months:
        print(f"- {month}")


In [ ]:
income['дата'].min()

In [ ]:
income['дата'].max()

In [ ]:
employees_with_missions = income['код_сотрудника'].unique()
employees_with_purchases = purs['код_сотрудника'].unique()

all_employees = set(employees_with_missions) | set(employees_with_purchases)
percent_with_missions = len(employees_with_missions) / len(all_employees) * 100
percent_with_purchases = len(employees_with_purchases) / len(all_employees) * 100

print(f"📌 Процент людей с хотя бы одной миссией: {percent_with_missions:.2f}%")
print(f"🛒 Процент людей с хотя бы одной покупкой: {percent_with_purchases:.2f}%")

In [153]:
purs['наименование_товара'].value_counts().head(8)

наименование_товара
Кофе КРОК                                4038
Кофе коллеге                             1692
Лотерейный билет "Надо собраться"        1406
Внешний аккумулятор Xiaomi                810
Подписка Яндекс плюс                      788
Лотерея на Кофемашину                     752
 Welcome-набор с кружкой и футболкой      722
 Welcome-набор с чехлом и футболкой       602
Name: count, dtype: int64

In [155]:
# Сумма всех покупок по всем сотрудникам
purs['сумма'] = purs['стоимость_в_валюте'] * purs['количество'] * (1 - purs['скидка']/100)

total_spent = purs['сумма'].sum()

# Количество уникальных сотрудников, которые делали покупки
unique_buyers = purs['код_сотрудника'].nunique()

# Средняя трата на одного покупающего сотрудника
avg_spent_per_person = total_spent / unique_buyers

print(f"💸 Средняя сумма, потраченная одним сотрудником: {avg_spent_per_person:.2f}")


💸 Средняя сумма, потраченная одним сотрудником: 8116.20
